<a href="https://colab.research.google.com/github/martinovzky/Pytorch-Classifier/blob/main/Pytorch_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries, Hyperparameters

In [2]:
#libraries

import torch as torch
import torch.nn as nn         # classes and functions for NN building
import torch.optim as optim   # optimization algorithms for training
from torch.utils.data import Dataset, DataLoader
import torchvision            # pre-trained models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder # for batching and loading data
import timm                                  # contains the models

import matplotlib.pyplot as plt              # for data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [3]:
#hyperparameters

batch = 32               # of images / training batch
number_epochs = 8       # tot number of training epochs (full passes over the dataset)
LearningRate = 1e-3      # learning rate for the optimizer (how much should model weights be updated after each batch)
#WeightDecay = 1e-1       # discourages large parameters in order not to overfit
Gamma = 0.1              # factor by which the learning rate will be reduced, helpw the model converge more smoothly and avoid overshooting local minima
NumClasses = 67          # MIT‑67 dataset has 67 scene categories
device = "cuda" if torch.cuda.is_available() else "cpu"


# Data Preparation


In [ ]:
import zipfile
import os
from google.colab import drive

drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/Colab_Notebooks/train'
test_dir = '/content/drive/MyDrive/Colab_Notebooks/test'


In [ ]:
# transformations for training and validation

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),             # resizes images to 224x224 pixels
    transforms.RandomHorizontalFlip(),         # randomly flips images horizontally for data augmentation
    transforms.RandomRotation(10),             # random rotation up to 10 degrees
    transforms.ColorJitter(                    # random changes in brightness/contrast/saturation/hue
        brightness=0.2, contrast=0.2,
        saturation=0.2, hue=0.1
    ),
    transforms.ToTensor(),                     # converts PIL Image to a PyTorch tensor (scales pixels to [0,1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # normalizes using ImageNet statistics
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),             # resizes images for consistency
    transforms.ToTensor(),                     # converts images to tensor, pixel values now between [0,1].
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # uses same normalization as training
                         std=[0.229, 0.224, 0.225])
])


# assigns labels to images based on subfolder names (= class names),  stores a list of image file paths and their corresponding labels.
train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = ImageFolder(root=test_dir, transform=val_transforms)

# DataLoaders to handle batching and shuffling. When iterated over, it loads batches of images and labels.
train_loader = DataLoader(train_dataset, batch_size= batch, shuffle=True, num_workers=2) # shuffle in order not to get overfitting
val_loader   = DataLoader(val_dataset, batch_size= batch, shuffle=False, num_workers=2) #num_workers = 2: 2 subprocesses to speed things up



# Model Setup


In [ ]:
#download model with pretrained weights
model = timm.create_model("efficientnet_b0", pretrained=True)

#adapts the model to classify images into NumClasses instead of the default 1000 ImageNet categories

in_features = model.classifier.in_features             # number of input features for the classifier layer
model.classifier = nn.Linear(in_features, NumClasses)  # replaces with a new linear layer

model = model.to(device)

In [ ]:
#loss function
criterion = nn.CrossEntropyLoss()

#optimizer updates model weights during training
optimizer = optim.Adam(model.parameters(), lr=LearningRate)

# learning rate schedule, reduces LR after every 5 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=Gamma)
# after every 4 epochs, the LR is multiplied by gamma=0.1

# Training & Validation

In [ ]:
#we train over 8 epochs

train_losses = []
val_losses = []

for epochs in range(number_epochs):
  model.train() #set the model to training mode: activates dropout (prevents overfitting), batchnorm, etc.
  running_loss = 0.0 #loss per epoch
  correct_samples = 0  #i.e. predicted label matches with real label
  total_sampled = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device) #moves these batches (tensors) them where the model is
    optimizer.zero_grad() #sets the gradients from previous steps to 0

    output = model(images) #forward pass, computes predictions (confidence scocres, not probabilities), and stores them in a tensor
    loss = criterion(output, labels) #computes loss between label predictions and actual labels

    #backprobagation : computes gradients of loss wrt the model's weights,this traverses the model backwards and probagates the outpout back to the input
    loss.backward()

    optimizer.step() #updates model weights

    # model performance

    running_loss += loss.item() * images.size(0)  #acumulated loss (loss per batch * number of images)
    _, preds = torch.max(output, 1)  #gets predicted class (preds) by taking the sample's max confidence score (_, is then ignored in order to later compute element-wise comparaison)
    correct_samples += (preds == labels).sum().item()  #counts correct predictions
    total_sampled += labels.size(0)  #counts total samples

  epoch_loss = running_loss / total_sampled  #average loss for the epoch
  train_losses.append(epoch_loss)

  epoch_acc  = correct_samples / total_sampled         #accuracy for the epoch

  print(f"Epoch {epochs+1}/{number_epochs} | Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")



  model.eval() #sets to validation mode
  val_loss = 0.0
  val_correct = 0
  val_total = 0

  #disables gradient calculations for validation to save memory and computation

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      output = model(images)           #forward pass
      loss = criterion(output, labels) #validation loss

      val_loss += loss.item() * images.size(0)  #acumulated loss (loss per batch * number of images)
      _, preds = torch.max(output,1)
      val_correct += (preds == labels).sum().item() # item() converts tensor to a scalar
      val_total += labels.size(0)

  val_loss = val_loss / val_total #average validation loss
  val_losses.append(val_loss)
  val_acc = val_correct / val_total #average validation accuracy

  print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.4f}")


# Data visualization

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()


In [ ]:
#saves the model's state dictionary (weights) to a file
torch.save(model.state_dict(), "efficientnet_b0_mit67.pth")
print("Model saved as efficientnet_b0_mit67.pth")
